In [1]:
import torch
import matplotlib.pyplot as plt
import torchvision
from torchvision.transforms import Resize
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, Dataset
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from nets import Classifier
import pickle
import numpy as np

In [2]:
class ImageDataSet(Dataset):
    def __init__(self, x, y, transform=None):
        self.x = x
        self.y = y
        self.transform = transform

    def __len__(self):
        return len(self.x)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()

        x = self.x[idx]
        y = self.y[idx]

        if self.transform:
            x = self.transform(x)

        return x, y

In [3]:
def shuffle_and_split(data, ratio):
    shuffle_idxs = torch.randperm(data[0].shape[0])

    img_shuffled = data[0][shuffle_idxs]
    label_shuffled = data[1][shuffle_idxs]

    split_idx = int(ratio * data[0].shape[0])

    train = (img_shuffled[:split_idx], label_shuffled[:split_idx])
    test = (img_shuffled[split_idx:], label_shuffled[split_idx:])

    return train, test

In [4]:
with open("/home/simon/catkin_ws/src/turtlebot3_gazebo/scripts/data/shapes_fixed_rgb8_1_10k_norm_np.pkl", 'rb') as f:
    squares_pose, squares_img = pickle.load(f)
with open("/home/simon/catkin_ws/src/turtlebot3_gazebo/scripts/data/shapes_dif_fixed_rgb8_1_10k_norm_np.pkl", 'rb') as f:
    circles_pose, circles_img = pickle.load(f)

squares_label = np.zeros(squares_img.shape[0])
circles_label = np.ones(circles_img.shape[0])

img = np.append(squares_img, circles_img, 0)
label = np.append(squares_label, circles_label)
data = (img, label)

train, test = shuffle_and_split(data, 0.7)

classes = ('squares', 'circles')

In [5]:
batch_size = 100

train_set = ImageDataSet(torch.tensor(train[0]).float(), torch.tensor(train[1]).float())
test_set = ImageDataSet(torch.tensor(test[0]).float(), torch.tensor(test[1]).float())

train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_set, batch_size=int(batch_size/10), shuffle=True)

In [30]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

print(device)

net = Classifier()
net.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

train_loss = []
test_loss = []

for epoch in range(100):  # loop over the dataset multiple times
    net.train()
    train_loss_batch = []
    for i, data in enumerate(train_loader):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data[0].to(device), data[1].to(device).to(torch.int64)

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        train_loss_batch.append(loss.item())

    if epoch%1==0:
        net.eval()
        test_loss_batch = []
        for i, data in enumerate(test_loader):
            inputs, labels = data[0].to(device), data[1].to(device).to(torch.int64)

            optimizer.zero_grad()  # zero the gradient buffers
            output = net(inputs)
            loss = criterion(output, labels)

            test_loss_batch.append(loss.item())
        
        test_loss.append(np.mean(test_loss_batch))

    train_loss.append(np.mean(train_loss_batch))
    print('EPOCH {}, TRAIN LOSS: {:.3f}, TEST LOSS: {:.3f}'.format(epoch+1, train_loss[-1], test_loss[-1]))

cuda:0
EPOCH 1, TRAIN LOSS: 0.691, TEST LOSS: 0.687
EPOCH 2, TRAIN LOSS: 0.674, TEST LOSS: 0.649
EPOCH 3, TRAIN LOSS: 0.487, TEST LOSS: 0.207
EPOCH 4, TRAIN LOSS: 0.078, TEST LOSS: 0.027
EPOCH 5, TRAIN LOSS: 0.013, TEST LOSS: 0.006
EPOCH 6, TRAIN LOSS: 0.006, TEST LOSS: 0.003
EPOCH 7, TRAIN LOSS: 0.004, TEST LOSS: 0.002
EPOCH 8, TRAIN LOSS: 0.002, TEST LOSS: 0.001
EPOCH 9, TRAIN LOSS: 0.002, TEST LOSS: 0.001
EPOCH 10, TRAIN LOSS: 0.001, TEST LOSS: 0.001
EPOCH 11, TRAIN LOSS: 0.001, TEST LOSS: 0.001
EPOCH 12, TRAIN LOSS: 0.001, TEST LOSS: 0.001
EPOCH 13, TRAIN LOSS: 0.001, TEST LOSS: 0.001
EPOCH 14, TRAIN LOSS: 0.001, TEST LOSS: 0.000
EPOCH 15, TRAIN LOSS: 0.001, TEST LOSS: 0.000
EPOCH 16, TRAIN LOSS: 0.001, TEST LOSS: 0.000
EPOCH 17, TRAIN LOSS: 0.000, TEST LOSS: 0.000
EPOCH 18, TRAIN LOSS: 0.000, TEST LOSS: 0.000
EPOCH 19, TRAIN LOSS: 0.000, TEST LOSS: 0.000
EPOCH 20, TRAIN LOSS: 0.000, TEST LOSS: 0.000
EPOCH 21, TRAIN LOSS: 0.000, TEST LOSS: 0.000
EPOCH 22, TRAIN LOSS: 0.000, TEST LO

In [7]:
torch.save(net.state_dict(), "context_classifier_2_100e.pt")

In [9]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

print(device)

net = Classifier()
net.to(device)
net.load_state_dict(torch.load("/home/simon/catkin_ws/src/turtlebot3_gazebo/scripts/trained_conv_nets/classifier_1_100e.pt"))

cuda:0


<All keys matched successfully>

In [10]:
correct = 0
total = 0

with torch.no_grad():
    for img, label in test_loader:
        out = net(img.to(device)).cpu()
        _, pred = torch.max(out, 1)
        total += label.size(0)
        correct += (pred == label).sum().item()

100*correct/total

8.683333333333334